In [13]:
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import pandas as pd
from datetime import datetime
from google.oauth2.service_account import Credentials
import gspread
from google.cloud import storage
from datetime import datetime
from pathlib import Path
import asyncio

In [14]:
data = {
    "userid": [
    "10002536", "10008847", "10004679", "10023836",
    "10033752", "10038584", "10044023", "10192623",
    "10165926", "10093976", "10132431"
    ],
    "名前": [
        "キンシュウサイ", "大岡正樹", "池海龍（イケカイリュウ）", "飯島美桜",
        "江原ケイト", "樋口大輔", "加藤順平", "加藤奈津子",
        "松本明子", "石崎卓", "玉野利家"
    ],
    "PW": [
        "Kin20240301", "Hasegawa110", "Chihailong0803", "Cherry2525",
        "Tennis2784!", "46495963¥@Desu", "WCbifd@3NANwXGE", "aDA8a6MWs3XEz_Y",
        "Saikou1234", "Zaitaku2023", "Packen0731t"
    ]
}

user_df = pd.DataFrame(data)

COLUMNS = ["userid","予約番号", "利用日", "時間", "公園・施設", "設備予約", "支払状況"]

In [ ]:
URL = "https://kouen.sports.metro.tokyo.lg.jp/web/index.jsp"
BLOCK_TEXT = "しばらく経ってから"
result_lst = []
async def scrape_cancel_list(user_df):
    p = await async_playwright().start()

    context = await p.chromium.launch_persistent_context(
        user_data_dir="./check_reservation",
        channel="chrome",
        headless=False,
    )
    page = await context.new_page()
    # page.on("request", lambda r: print(">>", r.method, r.url))
    # page.on("response", lambda r: print("<<", r.status, r.url))
    for _, row in user_df.iterrows():
        userid = str(row["userid"])
        pw = str(row["PW"])

        print(f"Processing user: {userid}")

        await page.goto(URL)

        # --- wait until not blocked ---
        while True:
            content = await page.content()
            if BLOCK_TEXT not in content:
                break
            print("Blocked. Waiting 5 seconds...")
            await asyncio.sleep(5)
            await page.reload()

        await page.click("text=ログイン")
        await page.fill("#userId", userid)
        await page.fill("#password", pw)
        # await page.wait_for_timeout(1500)
        
        # Login often triggers navigation
        async with page.expect_navigation(wait_until="networkidle"):
            await page.click("#btn-go")

        await page.wait_for_timeout(1500)

        # If login failed, login form will still exist
        if await page.locator("#userId").count():
            result_lst.append([userid, "login_fail", "login_fail", "login_fail", "login_fail", "login_fail", "login_fail"])


        # 2) Open reservation modal (as you already did)
        await page.click('a[data-target="#modal-reservation-menus"]')
        await page.wait_for_selector("#modal-reservation-menus", state="visible")

        # 3) Trigger the cancel/reservation list action ONCE and wait for navigation (if it happens)
        try:
            async with page.expect_navigation(wait_until="networkidle", timeout=10000):
                await page.evaluate("doAction(document.form1, gRsvWGetCancelRsvDataAction)")
        except PlaywrightTimeoutError:
            # In some flows it may update without a full navigation
            await page.wait_for_load_state("networkidle")

        # 4) If no reservations, tbody won't exist -> skip
        tbody = await page.query_selector("#rsvacceptlist tbody")

        if not tbody:
            result_lst.append([userid, "", "", "", "", "", ""])
        else:
            rows = await page.evaluate("""
            [...document.querySelectorAll('#rsvacceptlist tbody tr')]
            .map(tr => [...tr.querySelectorAll('td')]
                .slice(0, 6)
                .map(td => td.innerText.trim()))
            .filter(r => r.length === 6)
            """)
            result_lst.extend([[userid, *r] for r in rows])

        await page.goto(URL)
    
    await context.close()
    await p.stop()
    return pd.DataFrame(result_lst, columns=COLUMNS)
final_df = await scrape_cancel_list(user_df)


Processing user: 10002536
Blocked. Waiting 5 seconds...
Processing user: 10008847


RuntimeError: Login failed for user 10008847

In [34]:
async def scrape_cancel_list(user_df):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False, channel="chrome")
        result_lst = []

        for _, row in user_df.iterrows():
            userid = str(row["userid"])
            pw = str(row["PW"])
            print("Processing user:", userid)

            context = await browser.new_context()   # <-- key
            page = await context.new_page()

            await page.goto(URL, wait_until="domcontentloaded")
            await page.click("text=ログイン")
            await page.fill("#userId", userid)
            await page.fill("#password", pw)
            await page.click("#btn-go")

            menu = page.locator('a[data-target="#modal-reservation-menus"]')
            await menu.wait_for(state="visible", timeout=60_000)
            await menu.click()

            # ...rest of scraping...

            await context.close()

        await browser.close()
        return pd.DataFrame(result_lst, columns=COLUMNS)
final_df = await scrape_cancel_list(user_df.tail(2))

Processing user: 10093976
Processing user: 10132431


In [33]:
user_df

,userid,名前,PW
0,10002536,キンシュウサイ,Kin20240301
1,10008847,大岡正樹,Hasegawa110
2,10004679,池海龍（イケカイリュウ）,Chihailong0803
3,10023836,飯島美桜,Cherry2525
4,10033752,江原ケイト,Tennis2784!
5,10038584,樋口大輔,46495963¥@Desu
6,10044023,加藤順平,WCbifd@3NANwXGE
7,10192623,加藤奈津子,aDA8a6MWs3XEz_Y
8,10165926,松本明子,Saikou1234
9,10093976,石崎卓,Zaitaku2023


In [ ]:
# Generate date string
today_str = datetime.today().strftime("%Y%m%d")

# Build path safely
base_path = Path.home() / "workspace" / "熊猫カンパニー" / "reservation"
base_path.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
file_path = base_path / f"reservations_{today_str}.xlsx"

# Export
final_df.to_excel(file_path, index=False)

import subprocess

subprocess.run(["paplay", "/usr/share/sounds/freedesktop/stereo/complete.oga"])



CompletedProcess(args=['paplay', '/usr/share/sounds/freedesktop/stereo/complete.oga'], returncode=0)